In [15]:
from pprint import pprint
import yaml


metadata =  """
fields:
  age:
    faker: [random_int, 0, 100]
  age2:
    faker: random_int
    min_value: 0
    max_value: 100
"""
parsed = yaml.safe_load(metadata)
pprint(parsed)

{'fields': {'age': {'faker': ['random_int', 0, 100]},
            'age2': {'faker': 'random_int', 'max_value': 100, 'min_value': 0}}}


In [19]:
import pandas as pd

df = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'Dave', 'Eve'],
    'age': [25, 30, 35, 40, 45],
    'is_student': [True, False, True, False, True]
})

type(df['is_student'].mean().item())
unique = df['name'].value_counts(normalize=True)
unique

Alice      0.2
Bob        0.2
Charlie    0.2
Dave       0.2
Eve        0.2
Name: name, dtype: float64

In [12]:
from faker import Faker
fake = Faker()

fake.pybool()

True

In [18]:
from collections import OrderedDict
ordered = OrderedDict({'a': 1, 'b': 2})
ordered.popitem(0)

('a', 1)